2016-7-7 16:36:30 肌电项目验收之后, 杜宇的输出数据改为紧凑的csv, 放弃xml, good~
--
csv结构:
--
ax,ay,az,mx,my,mz,gx,gy,gz,rw,rx,ry,rz,node,frameid,timestamp,readtime,label

若要对比不同算法生成的轨迹(R,t), 请用 compareEulerAngle.ipynb 中的 diffCsvs
--

In [1]:
pwd

u'E:\\Github\\ipython.notebook.dir'

In [2]:
'拷贝自 depth-imu-calib-gen-synthetic.ipynb'

%pylab
import numbers
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import time
from datetime import datetime
from lxml import etree as et
from StringIO import StringIO
from primesense import openni2

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix #r也用a表示
tf.r2m = tf.rotation_matrix
tf.m2a = tf.m2r
tf.a2m = tf.r2m
tf.a2q = tf.quaternion_about_axis
tf.qmul = tf.quaternion_multiply
tf.qinv = tf.quaternion_inverse

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

def q2a(qx, qy, qz, qw, vecLen = 4):
    '''
    参见 tf.quaternion_about_axis 实现;
    算法参见:
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/angleToQuaternion/
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToAngle/
    
    测试代码：
    q=tf.a2q(0.123, (1,0,0))
    a=q2a(*q) #qxyzw顺序, 刻意与 tf 规则一致
    print q #[ 0.06146124  0.          0.          0.99810947]
    print a #(0.12300000000000065, 0.99999999999998979, 0.0, 0.0)
    '''
    if not ( isinstance(qx, numbers.Real) and isinstance(qy, numbers.Real) and isinstance(qz, numbers.Real) and isinstance(qw, numbers.Real) ):
        raise TypeError, 'Q:xyzw must be all numbers, not arrays / lists.'
        return
#     if not vecLen in (3, 4):
#         print 'vecLen 必须是 3 或者 4'
#         return
        
    angle = 2 * np.arccos(qw)
    'sin(angle/2):'
    sinA2 = np.sqrt(1 - qw * qw)
    x = qx / sinA2
    y = qy / sinA2
    z = qz / sinA2
    
    if vecLen == 4:
        return angle, x, y, z
    elif vecLen == 3:
        return x*angle, y*angle, z*angle
    else:
        raise ValueError, 'vecLen must be integer 3 or 4'
    pass

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from mpl_toolkits.mplot3d import axes3d

def plotAxAnim(qxyzws):
    '''
    关键帧mat矩阵不好看，改为3D绘制坐标轴，动画演示
    拷贝自 depth-imu-calib-gen-synthetic.ipynb, 改为函数, 2016-7-7 17:26:37
    qxyzws 必须是 local->global
    '''
    fig = figure()
    ax = fig.add_subplot(111, projection='3d')

    '添加xyz参考轴'
    o3d=(0,0,0)
    oxyz=np.eye(3)
    ax.plot_wireframe(*zip(o3d, oxyz[0]), color='r')
    ax.plot_wireframe(*zip(o3d, oxyz[1]), color='g')
    ax.plot_wireframe(*zip(o3d, oxyz[2]), color='b')

    wfx = wfy = wfz = None
    for i, q in enumerate(qxyzws):
    #     break
    #     print i
        oldwfx, oldwfy, oldwfz = wfx, wfy, wfz
    #     attribs = dat.attrib
    #     rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
    #     rmat = tf.q2m(rxyzw)[:3, :3]
        rmat = tf.q2m(q)[:3, :3]

        '调试看mat'
    #     if i==108:
    #         print i, rmat
    #     elif i==136:
    #         print i, rmat

    #     print(rmat)
        vx = rmat[:,0]
        vy = rmat[:,1]
        vz = rmat[:,2]
        wfx = ax.plot_wireframe(*zip((0,0,0), vx), color='r')
        wfy = ax.plot_wireframe(*zip((0,0,0), vy), color='g')
        wfz = ax.plot_wireframe(*zip((0,0,0), vz), color='b')
        if oldwfx != None:
            ax.collections.remove(oldwfx)
            ax.collections.remove(oldwfy)
            ax.collections.remove(oldwfz)
            pass

    #     fig.suptitle(str(i), fontsize=20)
    #     tit = ax.set_title(str(i))
        ax.title.set_text('idx: %s'%i)
        plt.pause(0.01)
        pass
    pass


In [18]:
'''
imu.csv; csv结构:ax,ay,az,mx,my,mz,gx,gy,gz,rw,rx,ry,rz,node,frameid,timestamp,readtime,label
'''
'绕xyz分别+360°, -360°'
fnImu = r'E:\zsens.0-1-2-13-gf27c3ff\bin\session-1191182355.csv'
'显示器-桌面, 缓慢转动, 同时采集oni; imu硬件朝下绑在kinect左下角'
fnImu = r'D:\Users\zhangxaochen\Documents\axxb-data\monitor-0708-0\monitor-0708-0-imu.csv'
'上面csv, 与 oni 帧同步(sync)之后的csv'
fnImu = r'D:\Users\zhangxaochen\Documents\axxb-data\monitor-0708-0\monitor-0708-0-imu-sync.csv'
# fnImu = r'D:\Users\zhangxaochen\Documents\axxb-data\monitor-0714-1\monitor-0714-1-imu-sync.csv'
# fnImu = r"D:\Users\zhangxaochen\Documents\axxb-data\monitor-0715-0\monitor-0715-0-imu-sync.csv"

datImu = genfromtxt(fnImu, dtype=None, delimiter=',', names=True)
print 'datImu.shape:', datImu.shape

# qwxyzs = array([dat['rw'], dat['rx'], dat['ry'], dat['rz']]).T
qwxyzsImu = datImu[['rw', 'rx', 'ry', 'rz']].view(float).reshape(-1,4)
qxyzwsImu = datImu[['rx', 'ry', 'rz', 'rw']].view(float).reshape(-1,4)

'由 Ri->g, 转到 Ri->R0'
q0 = qxyzwsImu[0]
qxyzwsImu = asarray([tf.qmul(tf.qinv(q0), qi) for qi in  qxyzwsImu])

'绘制四元数'
# %pylab inline
# plot(qxyzwsImu)

'imu绑在左下角时, imu->cam 理想旋转矩阵'
Ri2c = asarray([[0, -1, 0],
                [0,  0, 1],
                [-1, 0, 0]])
'''
axxb标定结果; 使用 monitor-0714-1-axxb130.csv
liang.m:
    0.0203   -0.9987    0.0463
   -0.0237    0.0458    0.9987
   -0.9995   -0.0214   -0.0227
park.m:
    0.0204   -0.9987    0.0461
   -0.0238    0.0456    0.9987
   -0.9995   -0.0215   -0.0228
高精度版:
[[0.0203938292269967, -0.998729549563985, 0.0460801319140112],
[-0.0237913773747099, 0.0455918836166632, 0.998676799826137],
[-0.999508910462108, -0.0214631539162760, -0.0228313585200759]]
'''
Ri2c = asarray([[0.0203938292269967, -0.998729549563985, 0.0460801319140112],
                [-0.0237913773747099, 0.0455918836166632, 0.998676799826137],
                [-0.999508910462108, -0.0214631539162760, -0.0228313585200759]])

t=eye(4)
t[:3,:3]=Ri2c
Ri2c=t

qi2c = tf.m2q(Ri2c)
print 'qi2c:', qi2c

qcamFromImus = asarray([tf.qmul(qi2c, tf.qmul(qi, tf.qinv(qi2c)) ) for qi in qxyzwsImu]) #xyzw

figure()
title('qcamFromImus')
plot( qcamFromImus) #确实跟下面 kf 绘制的很类似, 但是此处尚未做帧同步处理
ylim(ymax=1, ymin=-0.8)
legend(list('xyzw'))
# axis([335,355,-0.4,1])
# axis([0,300,-0.4,1])


datImu.shape: (1302,)
qi2c: [-0.49940798  0.51186654  0.47727951  0.51067464]


In [184]:
'问：理想旋转矩阵 vs. axxb标定矩阵偏差多少? //2016-7-15 17:32:29'
r1 = asarray([[0, -1, 0],
              [0,  0, 1],
              [-1, 0, 0]])
r2 = asarray([[0.0203938292269967, -0.998729549563985, 0.0460801319140112],
              [-0.0237913773747099, 0.0455918836166632, 0.998676799826137],
              [-0.999508910462108, -0.0214631539162760, -0.0228313585200759]])

dr = r1.dot(r2.T)
t=eye(4)
t[:3,:3]=dr
dr=t

angle, direct, point = tf.m2a(dr)
print 'angle, direct, point:', angle, direct, point
print 'angle°:', angle*r2a #3.18264179266°

angle, direct, point: 0.0555475781934 [ 0.41988078  0.37696077  0.82559113] [ 0.  0.  0.  1.]
angle°: 3.18264179266


In [17]:
'''
工具imu: 生成先前约定的 syntheticRT.txt (3+9=12列t+R格式, t在前); //2016-7-12 10:04:47
沿用上面 qcamFromImus, 即生成的 syntheticRT.txt 起始帧为单位阵
'''
import os

rmats = asarray([tf.q2m(qi)[:3,:3].flatten() for qi in qcamFromImus])
print rmats.shape
'fake伪造 t向量'
tvecs = zeros((rmats.shape[0], 3))
synMat = hstack((tvecs, rmats)) #输出矩阵

fnImuPath, _ = os.path.split(fnImu)
fnout = os.path.join(fnImuPath, 'syntheticRT.txt')
print 'fnout:', fnout
savetxt(fnout, synMat, delimiter=' ', fmt='%.10f')


(897, 9)
fnout: D:\Users\zhangxaochen\Documents\axxb-data\monitor-0715-0\syntheticRT.txt


In [7]:
'绘制四元数'
# plot(qxyzwsImu)
'轴角表示xyz+angle, 已验证capg-imu硬件坐标系定义是SEU, 2016-7-7 17:22:56'
agaxes = asarray([q2a(*q) for q in qxyzwsImu])
plot(agaxes[:, 1:]) #后三个量ax-xyz
plot(agaxes[:, 0]) #angle
legend(list('xyz')+['ang'])
'动画演示, 3D绘制坐标轴转动。//已验证capg-imu符合预期, 2016-7-8 10:22:17'
# plotAxAnim(qxyzwsImu)


'\xe5\x8a\xa8\xe7\x94\xbb\xe6\xbc\x94\xe7\xa4\xba, 3D\xe7\xbb\x98\xe5\x88\xb6\xe5\x9d\x90\xe6\xa0\x87\xe8\xbd\xb4\xe8\xbd\xac\xe5\x8a\xa8\xe3\x80\x82//\xe5\xb7\xb2\xe9\xaa\x8c\xe8\xaf\x81capg-imu\xe7\xac\xa6\xe5\x90\x88\xe9\xa2\x84\xe6\x9c\x9f, 2016-7-8 10:22:17'

In [4]:
# %pylab
'kf输出 txyz+qwxyz, w在前'
# fnKf = r'E:\Github\pcl\_build.vc10\bin\monitor-0708-0-kf\zcpose.csv'
fnKf = r'E:\Github\pcl\_build.vc10\bin\monitor-0708-0-kf-png\zcpose.csv'
# fnKf = r'E:\Github\pcl\_build.vc10\bin\monitor-0714-1-kf-png-d1-133\zcpose.csv'
# fnKf = r'E:\Github\pcl\_build.vc10\bin\monitor-0714-1-kf-png\zcpose.csv'
# fnKf = r'E:\Github\pcl\_build.vc10\bin\monitor-0714-1-pcc\zcpose.csv'
# fnKf = r'E:\Github\pcl\_build.vc10\bin\monitor-0714-1-pcc-imud\zcpose.csv'
# fnKf = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-kf\zcpose.csv"
fnKf = r'E:\Github\pcl\zctest\_build.vc10\synDep5\syn_pose.csv' #仿真数据, armadillo //2016-10-28 21:26:27

datKf = loadtxt(fnKf, delimiter=',')
print 'datKf.shape:', datKf.shape

tvecsKf = datKf[:,:3]
qwxyzKf = datKf[:,3:]
qxyzwKf = datKf[:,[4,5,6,3]]
figure()
# plot(qwxyzKf)
# legend(list('wxyz'))
title('kf')
plot(qxyzwKf)
ylim(ymax=1, ymin=-0.8)
legend(list('xyzw'))
# axis([0,155,-0.4,1])
# axis([0,300,-0.4,1])

'如果也要画 tvecs'
# plot(tvecsKf)

datKf.shape: (1495, 7)


'\xe5\xa6\x82\xe6\x9e\x9c\xe4\xb9\x9f\xe8\xa6\x81\xe7\x94\xbb tvecs'

In [168]:
'''
接上 一&三个cell, 用 qxyzwsImu, qxyzwKf 输出 axxb.m 需要的轴角表示的 csv, 格式:=(agaxCam, 000, agaxIMU, 000) (12列), //t全部伪造 000
kf用的就是相机坐标系, 不需担心;
输入的qxyzw物理意义：都是 i->0 (0视作global, 则是 l->g); kf=cam=A, imu=B;
输出的axag物理意义: cam: g->l(local); imu: l->g
输出csv格式: 空格分割 //参见: ir-cb-imu-precalib.oni.frames2calib.cvAndImuExtr.csv
2016-7-15 15:21:06
'''
'选用的帧序号'
inds = range(130)

kfAgax  = asarray([cv2.Rodrigues(tf.q2m(qi)[:3,:3].T)[0].ravel() for qi in qxyzwKf[inds]])
imuAgax = asarray([cv2.Rodrigues(tf.q2m(qi)[:3,:3])[0].ravel() for qi in qxyzwsImu[inds]])
t000 = zeros((kfAgax.shape[0], 3))
datCsv = hstack((kfAgax, t000, imuAgax, t000))
savetxt('axxb.csv', datCsv, delimiter=' ', fmt='%.10f')

In [76]:
'''
工具kf: 用 kf zcpose.csv 生成 syntheticRT.txt, 带不带真实t 都试试
'''

rmats = asarray([tf.q2m(qi)[:3,:3].flatten() for qi in qxyzwKf])
print rmats.shape
'fake伪造 t向量'
tvecs = zeros((rmats.shape[0], 3))
synMat = hstack((tvecs, rmats)) #输出矩阵

fnKfPath, _ = os.path.split(fnKf)
fnout = os.path.join(fnKfPath, 'syntheticRT.txt')
print 'fnout:', fnout
savetxt(fnout, synMat, delimiter=' ', fmt='%.8f')


(1302, 9)
fnout: E:\Github\pcl\_build.vc10\bin\monitor-0708-0-kf-png\syntheticRT.txt


In [80]:
rprev=[0.99972, -0.0236889, -0.00179956
, 0.0236051, 0.99904, -0.0369378
, 0.00267382, 0.0368848, 0.999321]
rinit=[0.99972, -0.0236889, -0.00179956
, 0.0236051, 0.99904, -0.0369378
, 0.00267382, 0.0368848, 0.999321]
rcurr=[0.999556,-0.0298614,0.00114073
,0.0298819,0.998887,-0.0365172,
-4.8032e-005,0.0365348,0.999338]
rprev=asarray(rprev).reshape(3,3)
t=eye(4)
t[:3,:3]=rprev
rprev=t
rinit=asarray( rinit).reshape(3,3)
t=eye(4)
t[:3,:3]=rinit
rinit=t
rcurr=asarray( rcurr).reshape(3,3)
t=eye(4)
t[:3,:3]=rcurr
rcurr=t
tf.m2r(rcurr.dot(rprev.T))

ValueError: no unit eigenvector corresponding to eigenvalue 1

In [92]:
randr=tf.random_rotation_matrix()
det(randr)

1.0

In [94]:
norm(tf.m2q(rprev))

1.0000010690246803

In [100]:
q=tf.m2q(rprev)
print q
aa=q2a(*q)
norm(aa[1:])


[ 0.01846008 -0.00111861  0.01182634  0.9997601 ]


1.0022258193763294

In [104]:
print q2a(*tf.m2q(rcurr.dot(rprev.T)))#[0]
print q2a(*tf.m2q(rcurr.dot(rinit.T)))#[0]


(0.0054172167594067075, -0.078084353848584892, 0.50053742841343996, 1.1589922886246966)
(0.0054172167594067075, -0.078084353848584892, 0.50053742841343996, 1.1589922886246966)


In [105]:
rprev

array([[ 0.99972   , -0.0236889 , -0.00179956,  0.        ],
       [ 0.0236051 ,  0.99904   , -0.0369378 ,  0.        ],
       [ 0.00267382,  0.0368848 ,  0.999321  ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [106]:
rinit

array([[ 0.99972   , -0.0236889 , -0.00179956,  0.        ],
       [ 0.0236051 ,  0.99904   , -0.0369378 ,  0.        ],
       [ 0.00267382,  0.0368848 ,  0.999321  ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [91]:
print det(rprev)
print det(rinit)
print det(rcurr)


1.00000855152
1.00000855152
1.00000905319


In [65]:
axis

<function matplotlib.pyplot.axis>

以上代码验证了
--
1. zsens-imu 姿态结果与 pcl_kinfu_app 姿态值在使用理想旋转矩阵之后大致一致 (注意: 设备重新绑定了)
2. 之前 imu 坐标轴定义理解正确;
3. 

以下代码用于处理
--
1. oni 与 imu.csv 数据帧同步，输出：oni->pngs, csv->new csv / imu-sync.csv
2. imu.csv 自己计算姿态 【未实现】

In [6]:
'''
数据组织到一个proj中去, 仿照 depth-imu-calib-gen-synthetic.ipynb //2016-7-10 16:49:02
新proj 包含基本: oni, begt.txt, imu.csv
'''

projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\monitor-0708-0'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\monitor-0714-1'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\monitor-0715-0'

os.chdir(projPath)

basePath, projName = os.path.split(projPath)
print 'projName:', projName
oniPath = projName+'.oni'
print 'oniPath:', oniPath
oniBegtPath = oniPath+'.txt' #记录oni起始绝对时间戳的txt
print 'oniBegtPath:', oniBegtPath
# f2cPath = oniPath+'.frames2calib.txt'
# print 'f2cPath:', f2cPath
# capXmlPath = 'CaptureSession.%s.xml'%projName
# print 'capXmlPath:', capXmlPath
# capFramesFolder = os.path.join(projPath, 'CapturedFrames.'+projName)
# extrFname = os.path.join(capFramesFolder, 'cam.xml')
# print 'extrFname:', extrFname
'增加 imu.csv'
imuCsvPath = projName + '-imu.csv'

if not os.path.exists(oniPath):
    print 'file not exist:', oniPath
if not os.path.exists(oniBegtPath):
    print 'file not exist:', oniBegtPath
if not os.path.exists(imuCsvPath):
    print 'file not exist:', imuCsvPath

'读起始时间begt'
from datetime import datetime
with open(oniBegtPath) as fin:
    begtEpoch = float(fin.read()) #epoch millis
    print "begtEpoch:", begtEpoch #1467965082147.0
    pass

'imu.csv 矩阵读入'
imuDat = genfromtxt(imuCsvPath, dtype=None, delimiter=',', names=True)

projName: monitor-0715-0
oniPath: monitor-0715-0.oni
oniBegtPath: monitor-0715-0.oni.txt
begtEpoch: 1.4685825888e+12


In [7]:
imuDat['readtime'][0]

'2016-07-15 19:36:19.058'

In [8]:
imuDat.shape

(1275,)

In [9]:
'''
oni 与 imu.csv 数据帧同步，输出：oni->pngs, csv->new csv
'''
from primesense import openni2
from datetime import datetime

openni2.initialize()     # can also accept the path of the OpenNI redistribution
dev = openni2.Device.open_file(oniPath)
print 'dev.get_device_info:', dev.get_device_info() #==dev.device_info
print 'dev.get_sensor_info:', dev.get_sensor_info(1)
'深度数据'
depth_stream = dev.create_depth_stream()
depth_stream.start()
'rgb 彩色数据'
color_stream = dev.create_color_stream()
color_stream.start()

depFcnt = depth_stream.get_number_of_frames()
print 'depFcnt:', depFcnt

oldDepfid = 0 #用于标记 fid 是否前进
imuFid = 0 #imu.csv 当前行指针
newImuCsvLst = []
depDatLst = []
for i in range(depFcnt):
    '读一帧, 期望不会跳帧, 但无法确保'
    depframe = depth_stream.read_frame()
    '真实帧序号'
    depfid = depframe.frameIndex #从 '1' 开始, 理论上应 = i+1, 即 oldDepfid == i
    #print 'i, depfid:', i, depfid
    if oldDepfid == depfid:
        print 'error?: 无新数据帧; oldDepfid == depfid =', depfid
    if oldDepfid + 1 < depfid:
        print 'error: 跳帧了, depfid =', depfid
    if i > depfid:
        print 'error: 帧循环了, depfid=', depfid
        
    oldDepfid = depfid #若无跳帧, 理论上应 oldDepfid == i
    
    '时间戳, 相对->绝对'
    depRelTs = depframe.timestamp #微秒，非毫秒
#     print 'depRelTs:', depRelTs
    
    depAbsTsMs = begtEpoch + depRelTs / 1e3
    depDtObj = datetime.fromtimestamp(depAbsTsMs / 1e3)
    
    for imuIdx in range(imuFid, imuDat.shape[0]):
        imuFid = imuIdx #无论时间戳比较结果, imuFid 必然前进
        imuCurrDt = datetime.strptime(imuDat['readtime'][imuIdx], '%Y-%m-%d %H:%M:%S.%f')
        if imuCurrDt >= depDtObj: #若 imu 时间戳超过当前深度帧的, 就记录 imu 前一帧
            #print 'imuCurrDt, depDtObj:', imuCurrDt, depDtObj
            break
#         else: #若 imu 时间戳还小
#             imuFid = imuIdx #等价于 imuFid+=1?
    
    if imuFid == 0:
        print 'error: imu.csv 第一帧时间戳就太大'
    
    newImuCsvLst.append(imuDat[imuFid - 1])
    
    '一帧数据矩阵'
    depdata = depframe.get_buffer_as_uint16()
#     depdata = asarray(depdata) #1D, dtype=uint16, 默认不 copy, 若要后处理就出错
    depdata = array(depdata) #1D, dtype=uint16, 不用 asarray, 因为要 copy=True
    depdata = depdata.reshape(depframe.height, depframe.width)
#     depFn = 'Depth_%04d.png'%depfid #保存图片的文件名
#     '保存图片文件'
#     cv2.imwrite(depFn, depdata)
    '写文件太慢, 可能导致跳帧, 移到外面, 另写循环'    
    depDatLst.append(depdata)
    
#     if i > 10: #短期调试用
#         break

newImuDat = asarray(newImuCsvLst)

'帧同步后, 写新的 imu.csv 文件'
imuCsvBasename, imuCsvExt = os.path.splitext(imuCsvPath)
newImuCsvPath = '%s-sync%s'%(imuCsvBasename, imuCsvExt)

# newImuDat.tofile(open('dummy.csv')) #IOError: XX requested and 0 written
# newImuDat.tofile('dummy.csv', sep=',', ) #不行

savetxt(newImuCsvPath, 
        newImuDat, 
        delimiter=',', fmt='%f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %d, %d, %d, %s, %d',
        comments='', #故意不用井号, 让 header 前面没有 '#'
        header='ax,ay,az,mx,my,mz,gx,gy,gz,rw,rx,ry,rz,node,frameid,timestamp,readtime,label')

print '写 dmap.png 图片序列...'
png_subpath = 'raw_frames'
os.makedirs(png_subpath) #无返回值, 若文件夹已存在, 则 WindowError

for i, dmat in enumerate(depDatLst):
    'png文件名, 这个序号未必等价于原 oni 中 fid, 且从0开始。存到 png_subpath=raw_frames 子文件夹中'
    depFn = '%s/Depth_%04d.png'%(png_subpath, i+1)
    cv2.imwrite(depFn, dmat)
    pass
print '写完了...'

'最后释放oni资源'
depth_stream.stop()
openni2.unload()

dev.get_device_info: OniDeviceInfo(uri = 'monitor-0715-0.oni', vendor = 'PrimeSense, Ltd.', name = 'oni File', usbVendorId = 0, usbProductId = 0)
dev.get_sensor_info: None
depFcnt: 897
写 dmap.png 图片序列...


In [111]:
'最后释放oni资源'
depth_stream.stop()
openni2.unload()

In [10]:
'播放图像序列'
for i, dmat in enumerate(depDatLst[:10]):
    'png文件名, 这个序号未必等价于原 oni 中 fid'
    depFn = 'Depth_%04d.png'%i 
    imshow(dmat) #热度图, 深蓝为0
    #matshow(depdata) #看起来与 imshow 一样?
    title(depFn)
    pause(0.1)
    pass

'cv2的方式, 原大小播放, 更好, 懒得用了'
# for i in range(1, 898):
#     fn=r"Depth_%04d.png"%i
#     img=cv2.imread(fn, -1)
#     cv2.putText(img, 'fid: %d'%i, (10, 111), cv2.FONT_HERSHEY_PLAIN, 4, 0, 2)
#     cv2.imshow('t', (img**1.5).astype(uint16))
#     cv2.waitKey()
    

d:\Anaconda2x86\lib\site-packages\matplotlib\backend_bases.py:2399: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [40]:
'工具: raw_frames 中 png 文件名数字统一 +1, 因为之前约定起始帧序号为 *1*; //2016-7-12 10:04:39'
import os
from glob import glob

rffn = r'D:\Users\zhangxaochen\Documents\axxb-data\monitor-0708-0\raw_frames'
os.chdir(rffn)

for fn in glob('*.png'):
    #id = int(fn.split('_')[1].split('.')[0])
    fn_head, t = fn.split('_')
    fn_id, ext = t.split('.')
    fn_id = int(fn_id) + 1 #文件名序号 +1
    new_fn = '%s_%04d.%s'%(fn_head, fn_id, ext)
    print fn, new_fn
    #os.rename(fn, new_fn) #从前往后导致重名 windowError, 放弃
    os.renames(fn, 'temp/'+new_fn) #放到 temp 路径；注意：会删除旧文件
    pass

Depth_0000.png Depth_0001.png
Depth_0001.png Depth_0002.png
Depth_0002.png Depth_0003.png
Depth_0003.png Depth_0004.png
Depth_0004.png Depth_0005.png
Depth_0005.png Depth_0006.png
Depth_0006.png Depth_0007.png
Depth_0007.png Depth_0008.png
Depth_0008.png Depth_0009.png
Depth_0009.png Depth_0010.png
Depth_0010.png Depth_0011.png
Depth_0011.png Depth_0012.png
Depth_0012.png Depth_0013.png
Depth_0013.png Depth_0014.png
Depth_0014.png Depth_0015.png
Depth_0015.png Depth_0016.png
Depth_0016.png Depth_0017.png
Depth_0017.png Depth_0018.png
Depth_0018.png Depth_0019.png
Depth_0019.png Depth_0020.png
Depth_0020.png Depth_0021.png
Depth_0021.png Depth_0022.png
Depth_0022.png Depth_0023.png
Depth_0023.png Depth_0024.png
Depth_0024.png Depth_0025.png
Depth_0025.png Depth_0026.png
Depth_0026.png Depth_0027.png
Depth_0027.png Depth_0028.png
Depth_0028.png Depth_0029.png
Depth_0029.png Depth_0030.png
Depth_0030.png Depth_0031.png
Depth_0031.png Depth_0032.png
Depth_0032.png Depth_0033.png
Depth_0033